# Spatiotemporal Trends in Urbanization: Cameroon
*Using yearly estimates (2000-2015) of population, built-area, and economic indicators to track city-by-city growth and change over time.*

---

### Research questions 

#### 1. How has the size of Settlement X changed over time? 

- Population size 

- Geographical extents 

- Population density 

#### 2. In what year did Settlement X become a new urban class?  

- From semi-dense to high-density city 

- Small settlement area to built-up area 

- When a hamlet area or small settlement area first appeared

#### 3. Is there a discernable pattern between the spatio-temporal distribution of economic density and population density? 

#### 4. How much of urban space attributable to City X is outside of the administrative limits of the city? 

- When did this fragment(s) appear? 

- Which district/municipality/authority has purview over the fragment(s)? 

#### 5. For the questions above, how does the answer change based on different understandings of urban limits? 

- Scenario A: where "city" is delimited by an official administrative boundary 

- Scenario B: where "city" includes all contiguous (and near-contiguous) built up area 

#### 6. Subnational and inter-national comparisons. Examples: 

- Compare the rates (pop, build-up, economic…) of the fastest growing settlement of each ADM1 region. 

- Which African metropoles experience the most vs. the least fragmentation? Is there a confluence between amount of urban fragmentation and rate of densification? 

### Datasets
1. Most up-to-date administrative boundaries: **ADM3.**
2. Built-up area, yearly: **World Settlement Footprint Evolution.** Resolution: 30m.
3. Settlement types: **GRID3 settlement extents.** Captured between 2009-2019.
4. Population, yearly: **WorldPop.** UN-adjusted, unconstrained. Resolution: 100m.
5. Nighttime lights, yearly: **Harmonization of DMSP and VIIRS.** Resolution: 1km.
6. City names: **UCDB, Africapolis, and GeoNames.**

---

---

## 1. PREPARE WORKSPACE

### 1.1 Off-script

##### Off-script: Create folders in working directory.
> *ADM
<br>Buildup
<br>PlaceName
<br>Population
<br>Settlement
<br>NTL*

##### Off-script: Download datasets (as shapefile, GeoJSON, or tif where possible) and place or extract into corresponding folder:
- ADM: *Sourced internally.*
- Buildup: https://download.geoservice.dlr.de/WSF_EVO/files/
- PlaceName: 
    - GeoNames: (file: cities500.zip) https://download.geonames.org/export/dump/
    - Africapolis: https://africapolis.org/en/data
    - Urban Centres Database: https://ghsl.jrc.ec.europa.eu/ghs_stat_ucdb2015mt_r2019a.php
- Population: https://hub.worldpop.org/geodata/listing?id=69
- Settlement: https://data.grid3.org/datasets/GRID3::grid3-cameroon-settlement-extents-version-01-01-/explore
- NTL: https://figshare.com/articles/dataset/Harmonization_of_DMSP_and_VIIRS_nighttime_light_data_from_1992-2018_at_the_global_scale/9828827/2

##### Other off-script:
- Convert GeoNames from .txt file to shape (delimiter = tab, header rows = 0) and rename fields.
- If necessary, mosaic WSFE rasters that cover the area of interest to create a single file.

### 1.2 Load all packages.

In [1]:
# Note: Most but not all of these packages were used in final form. 

import os, sys, glob, re, time
from os.path import exists
from functools import reduce

import geopandas as gpd 
import pandas as pd
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid, explain_validity
import shapely.wkt
import scipy

#from xrspatial import zonal_stats 
#import xarray as xr 
import numpy as np 
import fiona, rioxarray
import rasterio
from rasterio.plot import show
from rasterio import features
from rasterio.features import shapes
from rasterio import mask
from osgeo import gdal, osr, ogr, gdal_array
import matplotlib.pyplot as plt

### 1.3 Set workspace.

In [2]:
ProjectFolder = os.getcwd()
ResultsFolder = os.path.join(ProjectFolder, 'Results')
print(ProjectFolder)
print(ResultsFolder)

C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon
C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon\Results


---

## 2. PREPARE BUILDUP, SETTLEMENT, AND ADMIN DATASETS
Projection for all datasets: Africa Albers Equal Area Conic

### 2.1 WSFE: Check contents and change NoData value as necessary.

##### Off-script: Run this block in QGIS.

In [ ]:
# # OPEN QGIS FOR THIS PORTION. CODE DOCUMENTED HERE.
# Change NoData value to zero, as this won't interfere with a possible value of 99999 in GRID3 and ADM.
# Then make sure there are no values above 2015 (such as 99999) or below 1985 in the dataset by reclassifying them as NoData.
# Was having trouble with rasterio & gdal here, so moved to QGIS.

# processing.run("native:reclassifybytable", {'INPUT_RASTER':'C:/Users/grace/GIS/povertyequity/urban_growth/Cameroon/Buildup/WSFE_CMN.tif','RASTER_BAND':1,'TABLE':['2016','','0','','1984','0'],'NO_DATA':0,'RANGE_BOUNDARIES':0,'NODATA_FOR_MISSING':False,'DATA_TYPE':5,'OUTPUT':'C:/Users/grace/GIS/povertyequity/urban_growth/Cameroon/Buildup/WSFE.tif'})

### 2.2 Prepare raster locations for GRID3 and Admin areas

In [ ]:
ADM_vec = gpd.read_file(glob.glob('ADM/*.shp')[0])[['geometry']].to_crs("ESRI:102022") # This glob() function pulls the first file ([0]) in the ADM folder which ended in '.shp'
GRID3_vec = gpd.read_file(glob.glob('Settlement/*.shp')[0])[['type','geometry']].to_crs("ESRI:102022")
ADM_vec['ADM_ID'] = range(0,len(ADM_vec))
GRID3_vec['G3_ID'] = range(0,len(GRID3_vec))
ADM_vec.to_file(driver='GPKG', filename=r'ADM/ADM_warp.gpkg', layer='ADM')
GRID3_vec.to_file(driver='GPKG', filename=r'Settlement/Settlement_warp.gpkg', layer='GRID3')

In [ ]:
ADM_vec = gpd.read_file(r'ADM/ADM_warp.gpkg', layer='ADM')
GRID3_vec = gpd.read_file(r'Settlement/Settlement_warp.gpkg', layer='GRID3')

print(ADM_vec.info(), "\n\n", 
      ADM_vec.sample(5),
      ADM_vec.crs, "\n\n", 
      len(str(ADM_vec['ADM_ID'].max()))) # We need to know how many digits need to be allocated to each dataset in the "join" serial.
print(GRID3_vec.info(), "\n\n",
      GRID3_vec.sample(5),
      GRID3_vec.crs, "\n\n", 
      len(str(GRID3_vec['G3_ID'].max())))

---

## 3. WSFE AND ADM; GRID3 AND ADM
RASTERIZE: Bring ADM and GRID3 into raster space.

RASTER MATH: "Join" ADM ID onto GRID3 and onto WSFE by creating unique concatenation string.

VECTORIZE: Bring joined data into vector space.

VECTOR MATH: Split unique ID from raster math step into separate columns.

### 3.1 Reproject WSFE to project CRS.

In [ ]:
WSFE_in = glob.glob('Buildup/*.tif')[0]
WSFE_warp = './Buildup/WSFE_warp.tif'
ProjCRS = gdal.WarpOptions(dstSRS='ESRI:102022')

In [ ]:
Warp = gdal.Warp(WSFE_warp, # Where to store the warped raster
                 WSFE_in, # Which raster to warp
                 format='GTiff', 
                 options=ProjCRS) # Reproject to Africa Albers Equal Area Conic
Warp = None
print('Reprojected dataset. %s' % time.ctime())

try:  
    os.remove(os.path.join(ProjectFolder, WSFE_in))
except OSError:
    pass
print('Removed (or skipped if error) intermediate file. %s' % time.ctime())

In [ ]:
WSFE = rasterio.open(os.path.join(ProjectFolder, "Buildup", os.listdir('Buildup/')[0]))
print(WSFE) # WSFE values are all 4 digits long (1985-2015)
print(dir(WSFE))
print(WSFE.crs)
print(WSFE.dtypes)
NoDataValue = WSFE.nodatavals
print(NoDataValue)
print(WSFE.read(1).min(), WSFE.read(1).mean(), np.median(WSFE.read(1)), WSFE.read(1).max())

# If NoDataValue != 0, change to 0. (See step 2.1)

### 3.2 Rasterize admin areas and GRID3 using WSFE specs.

In [ ]:
# Copy and update the metadata from WSFE for the output
meta = WSFE.meta.copy()
meta.update(compress='lzw')
WSFE.meta

ADM_out = './ADM/ADM_rasterized.tif'
GRID3_out = './Settlement/GRID3_rasterized.tif'

In [ ]:
print("Rasterizing dataset. %s" % time.ctime())
with rasterio.open(ADM_out, 'w+', **meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(ADM_vec.geometry, ADM_vec.ADM_ID))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)
out = None

In [ ]:
print("Rasterizing dataset. %s" % time.ctime())
with rasterio.open(GRID3_out, 'w+', **meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(GRID3_vec.geometry, GRID3_vec.G3_ID))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)
out = None

*Validation: Check the dimensions, type, and basic stats of the three datasets. All should be the same dimension and NoData value.*

In [ ]:
RastersList = [gdal.Open(r"ADM/ADM_rasterized.tif"), 
               gdal.Open(r"Settlement/GRID3_rasterized.tif"),
               gdal.Open(os.path.join(ProjectFolder, "Buildup", os.listdir('Buildup/')[0]))]

for item in RastersList:
    print(gdal.GetDataTypeName(item.GetRasterBand(1).DataType), 
          item.GetRasterBand(1).GetNoDataValue(),
         "\n\n")

RastersList = None

RastersList = [rasterio.open(r"ADM/ADM_rasterized.tif"), 
               rasterio.open(r"Settlement/GRID3_rasterized.tif"), 
               rasterio.open(os.path.join(ProjectFolder, "Buildup", os.listdir('Buildup/')[0]))]

for item in RastersList:
    print(item.name, "\nBands= ", item.count, "\nWxH= ", item.width, "x", item.height, "\n\n")

stats = []
for item in RastersList:
    band = item.read(1)
    stats.append({
        'raster': item.name,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

# Show stats for each channel
print("\n", stats)

RastersList = None
band = None

### 3.2 Raster math to "join" admin to GRID3 and to WSFE.
Processing is more rapid when "joining," i.e. creating serial codes out of two datasets, in raster rather than vector space.
Here, we are concatenating the ID fields of the two datasets to create a serial number that we can then split in vector space later to create two ID fields.

*Adding together the values to create join IDs. This is in effect a concatenation of their ID strings, by way of summation. The number of zeros in the calc multiplication corresponds with number of digits of the maximum value in the "B" dataset. (e.g. Chad ADM codes go up 4 digits, so it's calc=(A*10000)+B).*

In [ ]:
# # OPEN TERMINAL FOR THIS PORTION. CODE DOCUMENTED HERE.

# Gdal_calc.py # To see info.

# gdal_calc.py -A C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon\Settlement\GRID3_rasterized.tif -B  C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon\ADM\ADM_rasterized.tif --outfile=C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon\Settlement\GRID3_ADM.tif --overwrite --calc="(A*1000)+B"
# gdal_calc.py -A C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon\Buildup\WSFE_warp.tif -B  C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon\ADM\ADM_rasterized.tif --outfile=C:\Users\grace\GIS\povertyequity\urban_growth\Cameroon\Buildup\WSFE_ADM.tif --overwrite --calc="(A*1000)+B"

# # END TERMINAL-ONLY ASPECT. RETURN HERE FOR NEXT STEPS.

In [ ]:
# Validation: check the basic statistics of the resulting datasets.
RastersList = [rasterio.open(r"Buildup/WSFE_ADM.tif"), 
               rasterio.open(r"Settlement/GRID3_ADM.tif")]
for item in RastersList:
    print(item.name, "\nBands= ", item.count, "\nWxH= ", item.width, "x", item.height, "\n\n")
    
stats = []
for item in RastersList:
    band = item.read(1)
    stats.append({
        'raster': item.name,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

# Show stats for each channel
print("\n", stats)

RastersList = None
band = None

### 3.3 Vectorize "joined" layers.

##### Off-script: Run this block in QGIS.

In [ ]:
# OPEN QGIS FOR THIS PORTION. CODE DOCUMENTED HERE.

# Due to dtype errors with both gdal and rasterio here, I decided to run the raster to polygon function in QGIS instead.
# It is possible to run QGIS functions within a Jupyter Notebook, but I ran it within the GUI. Arc or R are other options.
# Command line code here.

# processing.run("gdal:polygonize", {'INPUT':'C:/Users/grace/GIS/povertyequity/urban_growth/Cameroon/Settlement/GRID3_ADM.tif','BAND':1,'FIELD':'gridcode','EIGHT_CONNECTEDNESS':False,'EXTRA':'','OUTPUT':'C:/Users/grace/GIS/povertyequity/urban_growth/Cameroon/Settlement/GRID3_ADM.shp'})
# processing.run("gdal:polygonize", {'INPUT':'C:/Users/grace/GIS/povertyequity/urban_growth/Cameroon/Buildup/WSFE_ADM.tif','BAND':1,'FIELD':'gridcode','EIGHT_CONNECTEDNESS':False,'EXTRA':'','OUTPUT':'C:/Users/grace/GIS/povertyequity/urban_growth/Cameroon/Buildup/WSFE_ADM.shp'})

### 3.4 Vector math to split raster strings into admin area, GRID3, and WSFE year assignments.

In [ ]:
# Load newly created vectorized datasets.
GRID3_ADM = gpd.read_file(r"Settlement/GRID3_ADM.shp")
WSFE_ADM = gpd.read_file(r"Buildup/WSFE_ADM.shp")
print(GRID3_ADM.info(), "\n\n", GRID3_ADM.sample(10), "\n\n", GRID3_ADM.crs, "\n\n", 
      WSFE_ADM.info(), "\n\n", WSFE_ADM.sample(10), "\n\n", WSFE_ADM.crs)

In [ ]:
print(GRID3_ADM['gridcode'].max(), WSFE_ADM['gridcode'].max())

In [ ]:
# Split serial back into separate dataset fields.
# For Burkina: WSFE and ADM: 4+3=7 digits. GRID3 and ADM: 6+3=9 digits.
GRID3_ADM['gridstring'] = GRID3_ADM['gridcode'].astype(str).str.zfill(9)
WSFE_ADM['gridstring'] = WSFE_ADM['gridcode'].astype(str).str.zfill(7)

GRID3_ADM['Sett_ID'] = GRID3_ADM['gridstring'].str[:-3].astype(int) # Remove the last 3 digits to get the GRID3 portion.
GRID3_ADM['ADM_ID'] = GRID3_ADM['gridstring'].str[-3:].astype(int) # Keep only the last 3 digits to get the ADM portion.
WSFE_ADM['year'] = WSFE_ADM['gridstring'].str[:-3].astype(int)
WSFE_ADM['ADM_ID'] = WSFE_ADM['gridstring'].str[-3:].astype(int)

print(GRID3_ADM.sample(10), WSFE_ADM.sample(10))

In [ ]:
# Dissolve any features that have the same G3 and ADM values so that we have a single unique feature per settlement.
# Note: we do NOT want to dissolve the WSFE features. Distinct features for noncontiguous builtup areas of the same year is necessary to separate them in the Near tool step.
GRID3_ADM = GRID3_ADM.dissolve(by=['Sett_ID', 'ADM_ID'], as_index=False)
print(GRID3_ADM.info(), GRID3_ADM.head())

In [ ]:
# Remove features where year, settlement, or admin area = 0.
# This was supposed to be resolved earlier with the gdal_calc NoDataValue parameter.

print("Before: WSFE %s and GRID3 %s\n" % (WSFE_ADM.shape, GRID3_ADM.shape))
WSFE_ADM = WSFE_ADM.loc[(WSFE_ADM["year"] != 0) & (WSFE_ADM["ADM_ID"] != 0)] # Since we change the datatype to integer, no need to include all digits. Otherwise, it would need to be: != '0000'
GRID3_ADM = GRID3_ADM.loc[(GRID3_ADM["Sett_ID"] != 0) & (GRID3_ADM["ADM_ID"] != 0)]
print("After: WSFE %s and GRID3 %s\n" % (WSFE_ADM.shape, GRID3_ADM.shape))

In [ ]:
# The Bounded_ID is our new unique settlement identifier for subsequent matching steps.
GRID3_ADM['Bounded_ID'] = GRID3_ADM.index
WSFE_ADM['WSFE_ID'] = WSFE_ADM.index
GRID3_ADM = GRID3_ADM[['Sett_ID', 'Bounded_ID', 'ADM_ID', 'geometry']]
WSFE_ADM = WSFE_ADM[['WSFE_ID', 'year', 'ADM_ID', 'geometry']]

In [ ]:
# Validation: 
# The first two printed numbers should be the same. There shouldn't be any GRID3 rows with matching Sett_ID and ADM_IDs.
# The latter two numbers should be different, and the first should be larger. We never dissolved WSFE by any column.

print(len(GRID3_ADM[['Sett_ID', 'ADM_ID']]),
      len(GRID3_ADM[['Sett_ID', 'ADM_ID']].drop_duplicates()),
      len(WSFE_ADM[['year', 'ADM_ID']]),
      len(WSFE_ADM[['year', 'ADM_ID']].drop_duplicates()))

In [ ]:
GRID3_ADM.to_file(
    driver='GPKG', filename='Settlement/GRID3_ADM.gpkg', layer='GRID3_ADM_cleaned')
WSFE_ADM.to_file(
    driver='GPKG', filename=r'Buildup/WSFE_ADM.gpkg', layer='WSFE_ADM_cleaned')

---

## 4. UNIQUE SETTLEMENTS FROM WSFE AND GRID3: TWO VERSIONS

Note that there are 2 versions here, so that we can create a fragmentation index:
1. **Boundless, aka boundary-agnostic settlements**: Unique settlements are linked to GRID3 settlement IDs. Administrative areas do not influence the extents of the settlement.
2. **Bounded, aka politically-defined settlements**: Settlements in the Boundless dataset which spread across more than one administrative area are split into separate settlements in the Bounded dataset. The largest polygon after the split is considered the "principal" settlement, and polygons in other admin areas are considered "fragments." By dividing the fragment area(s) of the Bounded settlement by the area of the Boundless settlement, we can acquire a fragmentation index for each locality.

### 4.1 BOUNDED SETTLEMENTS: Near Join by ADM group.

In [ ]:
print("Number of admin areas with GRID3 features: %s" % len(GRID3_ADM['ADM_ID'].unique().tolist()))
print("Number of admin areas with WSFE features: %s" % len(WSFE_ADM['ADM_ID'].unique().tolist()))
print("Number of admin areas where one dataset is observed but the other is not: %s" % (
    len(GRID3_ADM['ADM_ID'].unique().tolist()) - len(WSFE_ADM['ADM_ID'].unique().tolist())))

In [ ]:
ADM_IDs = sorted(GRID3_ADM['ADM_ID'].unique().tolist())
ADM_IDs

In [ ]:
# We're creating this field to help in removing duplicates from the sjoin_nearest, next section.
GRID3_ADM['G3_Area'] = GRID3_ADM['geometry'].area / 10**6

In [ ]:
# Create empty geodataframe to append onto using the dataframe whose geometry we want to retain.
Bounded = GRID3_ADM[0:0]
Bounded["year"] = pd.Series(dtype='int')
Bounded.info()

In [ ]:
for ID in ADM_IDs:
    WSFE_shard = WSFE_ADM.loc[WSFE_ADM['ADM_ID'] == ID]
    GRID3_shard = GRID3_ADM.loc[GRID3_ADM['ADM_ID'] == ID]
    WSFE_GRID3_shard = gpd.sjoin_nearest(WSFE_shard, 
                                         GRID3_shard, 
                                         how='inner',
                                         max_distance=500)
    Bounded = pd.concat([Bounded, WSFE_GRID3_shard])
    print('Completed near join in admin area %s. %s \n' % (ID, time.ctime()))
print('Completed near join for all ADMs. %s \n' % time.ctime())

del WSFE_shard, GRID3_shard, WSFE_GRID3_shard

In [ ]:
Bounded.sample(20)

In [ ]:
Bounded.info()

In [ ]:
# Remove WSFE features that did not match any GRID3 settlements.
Bounded = Bounded.loc[~Bounded['Sett_ID'].isna()]
Bounded.info()

In [ ]:
del GRID3_ADM, ADM_IDs

### 4.2 Remove duplicates: where buildup polygons intersected with more than one GRID3 settlement extent.
This happens when the first dataset (WSFE) intersects (distance = 0) with more than one feature of the second dataset (GRID3). More common for large cities. For example, Yaoundé, CMN has a large contiguous 1985 WSFE polygon which overlaps several small GRID3 features that are not Yaoundé.

In [ ]:
# The first number should always be zero. 
# The second tells us whether/how many WSFE polygons were duplicated by the Near join.

print(len(WSFE_ADM[WSFE_ADM.duplicated('WSFE_ID')]), len(Bounded[Bounded.duplicated('WSFE_ID')]))

In [ ]:
# If there are duplicate WSFE_IDs, then we need to choose between them.
# We'll pick the one that joined with the largest GRID3 polygon.
# To do that, we can just sort the dataframe by GRID3 areas, then drop_duplicates. 
# It will retain the first row of each WSFE_ID group.
Bounded = Bounded.sort_values('G3_Area', ascending=False).drop_duplicates(['WSFE_ID'])
Bounded.info()

In [ ]:
print(len(Bounded[Bounded.duplicated('WSFE_ID')]))

In [ ]:
# Now we can dissolve with the WSFE years, now that we can group them by their administratively split ID.
Bounded = Bounded.dissolve(by=['year', 'Bounded_ID'], as_index=False)
print(Bounded.info(), Bounded.sample(10))

In [ ]:
# Clean up and save to file.
Bounded = Bounded[['ADM_ID_left', 'year', 'Bounded_ID', 'Sett_ID', 'geometry']].rename(columns={"ADM_ID_left": "ADM_ID"})
Bounded = Bounded.astype({"ADM_ID":'int', "Bounded_ID":'int', "Sett_ID":'int', "year":'int'})
print(Bounded.sample(10))
Bounded.to_file(
    driver='GPKG', filename=r'Results/NonCumulativeSettlements.gpkg', layer='Settlements_Bounded')

In [ ]:
del WSFE_ADM

### 4.3 BOUNDLESS SETTLEMENTS: Dissolve features that were split by an ADM boundary.

In [ ]:
# Fragments of any bounded settlement will be combined into a single "boundless" settlement in this version.
# It is based on their "Sett_ID", which is a direct loan from the GRID3 settlement features.
Boundless = Bounded.dissolve(by=['year', 'Sett_ID'], as_index=False)
print(Boundless.info(), Boundless.sample(10))

In [ ]:
# Clean up and save to file.
Boundless.to_file(driver='GPKG', filename=r'Results/NonCumulativeSettlements.gpkg', layer='Settlements_Boundless')

---

## 5. CUMULATIVE ANNUALIZED SETTLEMENT EXTENTS
DISSOLVE BY YEAR SETS: Create separate feature layers of each cumulative year.

### 5.1 Define study years for each for loop.

In [ ]:
Boundless = gpd.read_file(r'Results/NonCumulativeSettlements.gpkg', layer='Settlements_Boundless')

def CreateList(r1, r2):
    return [item for item in range(r1, r2+1)]

CuStart, CuEnd = Boundless['year'].min(), Boundless['year'].max()
StudyStart, StudyEnd = 1999, Boundless['year'].max()

AllCuYears = CreateList(CuStart, CuEnd) # All years in the WSFE dataset
AllStudyYears = CreateList(StudyStart, StudyEnd) # All years for which there will be growth stats in the present study.
print(AllCuYears, '\n\n', AllStudyYears)

ReversedStudyYears = []
for i in AllStudyYears:
    ReversedStudyYears.insert(0,i)
ReversedStudyYears.remove(StudyEnd)
print('\n\n', ReversedStudyYears)

### 5.2 Starting with main Boundless dataset, create a cumulative area feature layer for each year.

In [ ]:
# For each year in the growth stats study, we are taking features from all years prior to and including that year, 
# dissolving those features, and exporting as its own file.

for item in AllStudyYears:
    print('Subsetting to cumulative area for year: %s. %s\n' % (item, time.ctime()))
    CuYearSet = Boundless[Boundless['year'].between(
        CuStart, item, inclusive=True)] # Inclusive parameter means we include the years 1985 and "item" rather than only between them.
    print('Dissolving so that each unique settlement (Sett_ID) has a single cumulative WSFE feature. %s\n' % time.ctime())
    CuYearDissolve = CuYearSet.dissolve(by='Sett_ID', 
                                        aggfunc={"year": "max", "ADM_ID":"min"}, # Though ADM_ID should be matching every time.
                                        as_index=False)
    print('Write to file. %s\n' % time.ctime())
    CuYearName = ''.join(['Cu', str(item), '_Boundless'])
    CuYearDissolve.to_file(driver='GPKG', filename=r'Results/CumulativeSettlements.gpkg', layer=CuYearName)
    del CuYearSet, CuYearDissolve
print("Done with all years in set. %s" % time.ctime())

##### Join area information from each cumulative layer onto the latest year dataset.

In [ ]:
# The latest year in the study contains all settlements. Merge all other years' areas onto this dataset.
SettAreas = gpd.read_file(r'Results/CumulativeSettlements.gpkg', layer=
                          ''.join(['Cu', str(StudyEnd), '_Boundless'])) 
SettAreas['Area2015'] = SettAreas['geometry'].area / 10**6
SettAreas = pd.DataFrame(SettAreas).drop(columns='geometry') # We have settlement IDs, so no need to join spatially!


for item in ReversedStudyYears:
    print("Loading cumulative layer for year %s. %s\n" % (item, time.ctime()))
    YearLayer = gpd.read_file(r'Results/CumulativeSettlements.gpkg', layer=''.join(['Cu', str(item), '_Boundless']))
    print("Adding area field and converting to non-spatial dataframe. %s\n" % (time.ctime()))
    AreaYearName = ''.join(['Area', str(item)])
    YearLayer[AreaYearName] = YearLayer['geometry'].area/ 10**6 
    YearLayer = pd.DataFrame(YearLayer)[['Sett_ID', AreaYearName]]
    print("Merging variables from %s onto our latest year (%s) via table join. %s\n" % (item, StudyEnd, time.ctime()))
    SettAreas = SettAreas.merge(YearLayer, how='left', on='Sett_ID')
print("Done merging annualized areas onto latest year geometries. Saving to file. %s\n" % (time.ctime()))


print(SettAreas.info())
SettAreas.to_csv(os.path.join(ResultsFolder, 'Areas%sto%s.csv' % (StudyStart, StudyEnd)))

In [ ]:
del SettAreas

### 5.3 Repeat for Bounded dataset.

In [ ]:
# Bounded = gpd.read_file(r'Results/NonCumulativeSettlements.gpkg', layer='Settlements_Bounded')

for item in AllStudyYears:
    print('Subsetting to cumulative area for year: %s. %s\n' % (item, time.ctime()))
    CuYearSet = Bounded[Bounded['year'].between(CuStart, item, inclusive=True)] # Inclusive parameter means we include the years 1985 and "item" rather than only between them.
    print('Dissolving so that each unique settlement (Bounded_ID) has a single cumulative WSFE feature. %s\n' % time.ctime())
    CuYearDissolve = CuYearSet.dissolve(by='Bounded_ID', 
                                        aggfunc={"year": "max", "ADM_ID":"min", "Sett_ID":"min"}, # Though ADM_ID and Sett_ID should be matching every time.
                                        as_index=False)
    print('Write to file. %s\n' % time.ctime())
    CuYearName = ''.join(['Cu', str(item), '_Bounded'])
    CuYearDissolve.to_file(driver='GPKG', filename=r'Results/CumulativeSettlements.gpkg', layer=CuYearName)
    del CuYearSet, CuYearDissolve
print("Done with all years in set. %s" % time.ctime())

In [ ]:
SettAreas = gpd.read_file(r'Results/CumulativeSettlements.gpkg', 
                          layer=''.join(['Cu', str(StudyEnd), '_Bounded']))
SettAreas['Area2015'] = SettAreas['geometry'].area / 10**6
SettAreas = pd.DataFrame(SettAreas).drop(columns='geometry')


for item in ReversedStudyYears:
    print("Loading cumulative layer for year %s. %s\n" % (item, time.ctime()))
    YearLayer = gpd.read_file(r'Results/CumulativeSettlements.gpkg', layer=''.join(['Cu', str(item), '_Bounded']))
    print("Adding area field and converting to non-spatial dataframe. %s\n" % (time.ctime()))
    AreaYearName = ''.join(['Area', str(item)])
    YearLayer[AreaYearName] = YearLayer['geometry'].area/ 10**6 
    YearLayer = pd.DataFrame(YearLayer)[['Bounded_ID', AreaYearName]]
    print("Merging variables from %s onto our latest year (%s) via table join. %s\n" % (item, StudyEnd, time.ctime()))
    SettAreas = SettAreas.merge(YearLayer, how='left', on='Bounded_ID')
print("Done merging annualized areas onto latest year geometries. Saving to file. %s\n" % (time.ctime()))

print(SettAreas.info())
SettAreas.to_csv(os.path.join(ResultsFolder, 'Areas%sto%s_%s.csv' % (StudyStart, StudyEnd, 'Bounded')))

In [ ]:
del SettAreas

### 5.4 One settlement geofile to rule them all. ...and in the Sett_ID bind them.
The annualized values can be stored as distinct non-spatial dataframes. Their Sett_IDs will be used to join onto this geoversion with place names for the summary stats.

In [ ]:
Settlements = gpd.read_file(r'Results/CumulativeSettlements.gpkg', 
                           layer=''.join(['Cu', str(StudyEnd), '_Boundless']))
print(Settlements.info())

In [ ]:
Settlements = gpd.read_file(r'Results/CumulativeSettlements.gpkg', 
                           layer=''.join(['Cu', str(StudyEnd), '_Boundless']))[['Sett_ID', 'ADM_ID', 'geometry']]
print(Settlements.info())
Settlements.to_file(driver='GPKG', 
                       filename=r'Results/SETTLEMENTS.gpkg', 
                       layer='SETTLEMENTS')

### 5.5 Buffer the area of the Boundless dataset's latest year to mask raster data in later sections.
The Bounded dataset would also be fine for our purposes here. The buffer is dissolved to a single feature to be used for its total extents, which are identical between Bounded & Boundless datasets.

In [ ]:
# Create buffer layer(s) to use as maximum distance for Near joins.

# Population buffer: 2km
Distance = 2000

print('Creating buffer layer. %s' % time.ctime())
BufferLayer = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS')
BufferLayer['geometry'] = BufferLayer['geometry'].apply(
    make_valid).buffer(Distance) # make_valid is a workaround for any null geometries.
print('Finished buffer layer creation. %s' % time.ctime())
BufferFileName1 = ''.join(['Buff', str(Distance), 'm_', str(StudyEnd)])
BufferLayer.to_file(driver='GPKG', filename=r'Results/Catchment.gpkg', layer=BufferFileName1)
print('Saved to file. %s' % time.ctime())

In [ ]:
# NTL buffer: 250m
Distance = 250

print('Creating buffer layer. %s' % time.ctime())
BufferLayer = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS')
BufferLayer['geometry'] = BufferLayer['geometry'].apply(
    make_valid).buffer(Distance) # make_valid is a workaround for any null geometries.
print('Finished buffer layer creation. %s' % time.ctime())
BufferFileName2 = ''.join(['Buff', str(Distance), 'm_', str(StudyEnd)])
BufferLayer.to_file(driver='GPKG', filename=r'Results/Catchment.gpkg', layer=BufferFileName2)
print('Saved to file. %s' % time.ctime())

---

## 6. PLACE NAMES
Join urban place names from UCDB, Africapolis, and GeoNames onto the settlement vectors.

### 6.1 Load placename datasets, filter, and project.

In [ ]:
# If restarting here:
Settlements = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS')
Settlements['Area2015'] = Settlements['geometry'].area / 10**6

# Load, pull name field, rename, and reproject to match the catchments CRS.
UCDB = gpd.read_file('PlaceName/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg', 
                     layer=0)[['UC_NM_MN', 'geometry']].rename(
    columns={"UC_NM_MN": "UCDB_Name"}).to_crs("ESRI:102022")

Africapolis = gpd.read_file('PlaceName/AFRICAPOLIS2020.shp')[['agglosName', 'geometry']].rename(
    columns={"agglosName": "Afpl_Name"}).to_crs("ESRI:102022")

GeoNames = gpd.read_file('PlaceName/GeoNames.gpkg', 
                         layer=0)[['GeoName', 'geometry']].to_crs("ESRI:102022")

print(Settlements.info(), UCDB.info(), Africapolis.info(), GeoNames.info())

### 6.2 Join placenames onto settlements geodataframe.

In [ ]:
# We wrap it in pd.DataFrame() since the sjoin() is the last time we need the geometry.

GeoNames = pd.DataFrame(gpd.sjoin_nearest(GeoNames, Settlements, 
                             how='left', distance_col="distGN", max_distance=250, 
                             lsuffix="G3", rsuffix="GN")).drop(columns='geometry')
Africapolis = pd.DataFrame(gpd.sjoin_nearest(Africapolis, Settlements, 
                             how='left', distance_col="distAF", max_distance=250,
                             lsuffix="G3", rsuffix="Af")).drop(columns='geometry')
UCDB = pd.DataFrame(gpd.sjoin_nearest(UCDB, Settlements, 
                             how='left', distance_col="distUC", max_distance=250,
                             lsuffix="G3", rsuffix="UC")).drop(columns='geometry')

In [ ]:
print(GeoNames.info())
print(Africapolis.info())
print(UCDB.info())

In [ ]:
alldatasets = [pd.DataFrame(Settlements).drop(columns='geometry'),
               Africapolis[['Sett_ID', 'Afpl_Name', 'distAF']], 
               GeoNames[['Sett_ID', 'GeoName', 'distGN']],
               UCDB[['Sett_ID', 'UCDB_Name', 'distUC']]]

SettlementsNamed = reduce(lambda left,right: pd.merge(left,right,on=['Sett_ID'], how='left'), alldatasets)
SettlementsNamed[['Afpl_Name', 'GeoName', 'UCDB_Name']] = SettlementsNamed[['Afpl_Name', 'GeoName', 'UCDB_Name']].fillna('UNK')

# Replace NaN values with a countable distance.
SettlementsNamed[['distAF', 'distGN', 'distUC']] = SettlementsNamed[['distAF', 'distGN', 'distUC']].fillna(-1)

In [ ]:
print(SettlementsNamed.info())
print(SettlementsNamed.sample(10))

In [ ]:
del UCDB, Africapolis, GeoNames

The near joins should have prevented duplication of rows, but if df1 intersects with two features in df2, it creates a new row. Two of our placenames sources are polygons, so there may be instances.

In [ ]:
SettlementsNamed[SettlementsNamed.duplicated('Sett_ID', keep=False)]

In [ ]:
SettlementsNamed.drop_duplicates(subset=['Sett_ID'], inplace=True, keep='first')
SettlementsNamed.info() # Range of entries should be the same as original Settlements file.

### 6.3 Reduce to single name column.

In [ ]:
# Determine which source has a name geometrically closest to the settlement.
# Since we switched NaN values to -1 earlier, we also resolved what happens in the event of a tie, 
# i.e. when more than one source is 0.0 meters from the settlement. It will take the value from the first column.
SettlementsNamed['SettName'] = "UNK"
SettlementsNamed['closest'] = SettlementsNamed[['distAF', 'distGN', 'distUC']].idxmax(axis=1)

In [ ]:
SettlementsNamed.sample(20)

In [ ]:
# Create a single name column where non-named settlements are "UNK" but all others use one of the three name sources.
SettlementsNamed.loc[
    SettlementsNamed['closest'] == "distAF", 
    'SettName'] = SettlementsNamed['Afpl_Name']

SettlementsNamed.loc[
    SettlementsNamed['closest'] == "distUC", 
    'SettName'] = SettlementsNamed['UCDB_Name']

SettlementsNamed.loc[
    SettlementsNamed['closest'] == "distGN", 
    'SettName'] = SettlementsNamed['GeoName']

In [ ]:
SettlementsNamed.sample(20)

### 6.4 Make sure place name is unique by stripping smaller localities of duplicated names.

In [ ]:
Dupes = SettlementsNamed[ 
    (SettlementsNamed['SettName'] != 'UNK') & 
    (SettlementsNamed.duplicated('SettName', keep=False)) ] # keep=False is necessary to retain *all* duplicates, not just first or last in each group.

print("Number of named settlements: %s" % SettlementsNamed['SettName'].str.contains('UNK').value_counts()[False])
print("Number of named settlements where name is duplicated at least once: %s" % len(Dupes))

In [ ]:
Largest = Dupes.loc[Dupes.groupby(["SettName"])["Area2015"].idxmax()]
print(Largest)

In [ ]:
# Filter to settlements which have a duplicated name and are not the largest of those with that name, then replace with UNK.
SettlementsNamed.loc[(~SettlementsNamed.Sett_ID.isin(Largest.Sett_ID)) 
                     & (SettlementsNamed.Sett_ID.isin(Dupes.Sett_ID)), 
                     'SettName'] = 'UNK'

In [ ]:
# Second number should now be zero.

print("Number of named settlements: %s" % SettlementsNamed['SettName'].str.contains('UNK').value_counts()[False])
print("Number of named settlements where name is duplicated at least once: %s" % len(SettlementsNamed[ 
    (SettlementsNamed['SettName'] != 'UNK') & 
    (SettlementsNamed.duplicated('SettName', keep=False)) ]))

In [ ]:
print(SettlementsNamed.info(), SettlementsNamed[SettlementsNamed['SettName'] != "UNK"].sample(20))

In [ ]:
# Drop extra columns and save to file.
SettlementsNamed = SettlementsNamed[['Sett_ID', 'SettName']]
SettlementsNamed.to_csv(r'Results/PlaceNames.csv')

In [ ]:
del SettlementsNamed

---

## 7. CREATE FRAGMENTATION INDEX
We are determining what percentage of a settlement's area lies outside of its administrative zone each year.
The index is a range of 0 to 100, i.e. the percent of the settlement area which is fragmented.

For each Sett_ID:
((Area of Boundless settlement - Area of largest Bounded settlement feature) / Area of Boundless settlement) * 100

### 7.1 Load boundless and bounded cumulative settlements and clean.

In [ ]:
BoundlessAreas = pd.read_csv(os.path.join(ResultsFolder, ('Areas%sto%s.csv' % (StudyStart, StudyEnd))))
print('Loaded Boundless dataset, whose settlements will be used as the index of the Fragmentation Index dataset. %s' 
      % time.ctime())
print(BoundlessAreas.info())

BoundedAreas = pd.read_csv(os.path.join(ResultsFolder, ('Areas%sto%s_%s.csv' % (StudyStart, StudyEnd, 'Bounded'))))
print('Loaded Bounded dataset, which will factor into the fragmentation calculation. %s' % time.ctime())
print(BoundedAreas.info())

In [ ]:
LargestFragments = BoundedAreas.loc[BoundedAreas.groupby(["Sett_ID"])["Area2015"].idxmax()] 
print(LargestFragments.info())
print("Filtered the Bounded dataset to only rows where latest year's area is largest for each Sett_ID. %s" % time.ctime())
LargestFragments.columns = LargestFragments.columns.str.replace('Area', 'Largest')
LargestFragments = LargestFragments.drop(columns=['year', 'ADM_ID'])
print("Renamed columns to avoid duplication during merge, and dropped unnecessary columns. %s" % time.ctime())
FragIndices = BoundlessAreas.merge(LargestFragments, how='left', on='Sett_ID')
print(FragIndices.info())

In [ ]:
del BoundlessAreas, BoundedAreas, LargestFragments

### 7.2 Merge and run fragmentation calculation.

In [ ]:
for item in AllStudyYears:
    YY = str(item) # 4-digit year
    AreaYY = ''.join(["Area", YY]) # The Boundless area variable name
    LargestYY = ''.join(['Largest', YY]) # The Bounded largest area variable name
    FragYY = ''.join(["Frag", YY]) # Name for the fragmentation index variable
    print("Created names for Year %s's variables and temporary objects. %s" % (item, time.ctime()))
    
    FragIndices[FragYY] = ((FragIndices[AreaYY] - FragIndices[LargestYY]) / FragIndices[AreaYY]) * 100
    FragIndices[FragYY] = (FragIndices[FragYY].fillna(0).replace([np.inf, -np.inf], 0)).astype('int')
    print("Calculated fragmentation index for year %s. %s" % (item, time.ctime()))

# Remove unnecessary columns.
FragIndices = FragIndices.loc[:, ~FragIndices.columns.str.startswith('Largest')]
FragIndices = FragIndices.loc[:, ~FragIndices.columns.str.startswith('Area')]

print('Completed fragmentation index calculations for all years. %s' % time.ctime())
print(FragIndices.info())
print(FragIndices.sample(5))

In [ ]:
FragIndices = FragIndices.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'year', 'ADM_ID'])
FragIndices.to_csv(os.path.join(ResultsFolder, 'FragIndex%sto%s.csv' % (StudyStart, StudyEnd)))
print('Saved to file. %s' % time.ctime())

In [ ]:
del FragIndices

---

## 8. PREPARE YEARLY DATASETS: POPULATION
Can use this as a template for other annualized rasters

### 8.1 Reproject and reclassify with settlement buffer mask.
Reclassify so that we only need to work with cells within X distance of settlements.

In [ ]:
ProjCRS = gdal.WarpOptions(dstSRS='ESRI:102022')
AnnualizedSourceFiles = [i for i in os.listdir('Population/') if i.endswith('.tif')]

with fiona.open(r'Results/Catchment.gpkg', mode="r", layer="Buff2000m_2015") as shapefile:
    MaskGeom = [feature["geometry"] for feature in shapefile] # Identify the bounding areas of the mask.
# Mask_out = './LatestYearBuffer.tif'
AnnualizedSourceFiles

In [ ]:
# This codeblock changes each annual population raster's projection (gdal.Warp()), 
# then masks it to within a specified distance of the settlements (rasterio.mask.mask()).

for YearFile in AnnualizedSourceFiles:
    InputRasterName = os.path.join(ProjectFolder, "Population", YearFile)
    Year = str(re.sub(r'[^0-9]', '', YearFile))
    InputRasterObject = gdal.Open(InputRasterName)
    TempOutputName = "Temp_" + Year + "_albers.tif"
    TempOutputPath = os.path.join(ProjectFolder, "Population", TempOutputName)
    if exists(TempOutputPath):
        pass
    else:
        # Reproject to same CRS as settlements.
        Warp = gdal.Warp(TempOutputPath, # Where to store the warped raster
                     InputRasterObject, # Which raster to warp
                     format='GTiff', 
                     options=ProjCRS) # Reproject to Africa Albers Equal Area Conic
        print('Finished gdal.Warp() for year %s. %s \n' % (Year, time.ctime()))
        
        Warp = None # Close the files
        InputRasterObject = None

        # Reclassify as nodata if outside settlement buffer zones.
        with rasterio.open(TempOutputPath) as InputRasterObject:
            MaskedOutputRaster, OutTransform = rasterio.mask.mask(
                InputRasterObject, MaskGeom, crop=True) # Anything outside the mask is reclassed to the raster's NoData value.
            OutMetaData = InputRasterObject.meta.copy()
        print('Finished rasterio.mask.mask() for year %s. %s \n' % (Year, time.ctime()))
            
        OutMetaData.update({"driver": "GTiff",
                         "height": MaskedOutputRaster.shape[1],
                         "width": MaskedOutputRaster.shape[2],
                         "transform": OutTransform})
        FinalOutputPath = os.path.join(ProjectFolder, "Population", ''.join(['Masked_', Year, '.tif'])) # ''.join([r'Population/', 'Masked_', Year, '.tif']
        with rasterio.open(FinalOutputPath, "w", **OutMetaData) as dest:
            dest.write(MaskedOutputRaster)
        print('Written to file. %s \n' % time.ctime())
    InputRasterObject = None
    
    try:  # Finally, remove the intermediate file from disk
        os.remove(TempOutputPath)
    except OSError:
        pass
    print('Removed intermediate file. %s \n' % time.ctime())

print('\n \n Finished all years in list. %s' % time.ctime())

In [ ]:
print(os.listdir('Population/'))

In [ ]:
AnnualizedSourceFiles = None

### 8.2 Raster values summarized by settlement.
1. Convert each annualized raster to .xyz, 
2. then bring them to vector space and assign their Sett_ID,
3. and finally, aggregate the value as appropriate to the settlement level and save table to file.

XYZ is similar to .csv. Raster cell centers are stored as x and y, and their value is stored as z.

In [ ]:
NoDataVal = -99999 
Settlements = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS')
AllSummaries = pd.DataFrame(Settlements).drop(columns='geometry')

AnnualizedMaskedFiles = [i for i in os.listdir('Population/') if i.startswith('Masked') and i.endswith('.tif')]
AnnualizedMaskedFiles

In [ ]:
for YearFile in AnnualizedMaskedFiles:
    
### STEP 1: TIF TO XYZ ###
    InputRasterName = os.path.join(ProjectFolder, "Population", YearFile)
    Year = str(re.sub(r'[^0-9]', '', YearFile))
    print('Loading data for year %s. %s \n' % (Year, time.ctime()))
    InputRasterObject = gdal.Open(InputRasterName)
    XYZOutputPath = r'Population/{}'.format(
        YearFile.replace('.tif', '.xyz')) # New file path will be the same as original, but .tif is replaced with .xyz
    
    # Create an .xyz version of the .tif
    XYZ = gdal.Translate(XYZOutputPath, # Specify a destination path
                         InputRasterObject, # Input is the masked .tif file
                         format='XYZ', 
                         creationOptions=["ADD_HEADER_LINE=YES"])
    print('Finished gdal.Translate() for year %s. %s \n' % (Year, time.ctime()))

#     # Remove the temporary masked tif file.
#     try:  
#         os.remove(InputRasterName)
#     except OSError:
#         pass
#     print('Removed (or skipped if error) intermediate tif file. %s \n' % time.ctime())
    
    InputRasterObject = None
    XYZ = None # Reload XYZ as a point geodataframe

    
### STEP 2: GENERATE GEODATAFRAME WITH SETT_ID FIELD ###
    InputXYZName = ''.join(['Masked_', Year, '.xyz'])
    InputXYZ = pd.read_table(os.path.join(ProjectFolder, 'Population', InputXYZName), delim_whitespace=True)
    InputXYZ = InputXYZ.loc[InputXYZ['Z'] != NoDataVal] # Subset to only the features that have a raster value.
    print('Loaded XYZ file as a pandas dataframe, year %s. %s \n' % (Year, time.ctime()))
    ValObject = gpd.GeoDataFrame(InputXYZ,
                                 geometry = gpd.points_from_xy(InputXYZ['X'], InputXYZ['Y']),
                                 crs = 'ESRI:102022')
    print('Created geodataframe from non-NoData points, year %s. %s \n' % (Year, time.ctime()))
    del InputXYZ
    
    # Sjoin_nearest: No need to group by ADM this time. 
    ValObject_withID = gpd.sjoin_nearest(ValObject, 
                                    Settlements, 
                                    how='left') # No need for max_distance parameter this time. We've already narrowed down to nearby raster cells.
    
    print('\nJoined settlement ID onto vectorized raster cells for year %s. %s \n' % (Year, time.ctime()))
    print(ValObject_withID.sample(10))
    del ValObject
    
    # We no longer need the spatial information of the raster values because we have their unique settlement ID.
    ValObject_withID = pd.DataFrame(ValObject_withID).drop(columns='geometry')
    
    ValObject_withID.to_csv(''.join([r'Population/', 'Masked_', Year, '.csv']))
    print('\nExported as table, year %s. %s \n' % (Year, time.ctime()))
    
    # Remove the temporary xyz file.
    try:  
        os.remove(os.path.join(ProjectFolder, 'Population', InputXYZName))
    except OSError:
        pass
    print('Removed (or skipped if error) intermediate xyz file. %s \n' % time.ctime())

    

### STEP 3: AGGREGATE BY SETTLEMENT AND MERGE ONTO SUMMARIES TABLE ###
    VariableName = ''.join(['PopSum', Year])
    
    ValAggregated = ValObject_withID.groupby('Sett_ID', 
                                      as_index=False)['Z'].sum().rename(columns={"Z": VariableName})
    print('\nValues aggregated to settlement level, year %s. %s \n' % (Year, time.ctime()))
    print(ValAggregated.sample(10))
    
    AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
    print('\nMerged year %s onto latest year settlement feature layer. %s \n' % (Year, time.ctime()))
    print(AllSummaries.sample(10))
    
    del ValObject_withID, ValAggregated
    print('\n\n')
    

print('\n\nFinished. All years masked and assigned their nearest settlement. %s' % time.ctime())

AllSummaries.to_csv(os.path.join(ResultsFolder, 'Pop%sto%s.csv' % (2000, 2015)))
print('Saved to file. %s \n' % time.ctime())

In [ ]:
# Check contents
AllSummaries.sort_values('PopSum2010', ascending=False).head(20)

---

## 9. PREPARE YEARLY DATASETS: NIGHTTIME LIGHTS

### 9.1 Reclassify with settlement buffer mask.
Reclassify so that we only need to work with cells within X distance of settlements. The two NTL sources have already been reprojected in a separate script, and cropped to Central & Western Africa.

In [3]:
ProjCRS = gdal.WarpOptions(dstSRS='ESRI:102022')
with fiona.open(r'Results/Catchment.gpkg', mode="r", layer="Buff250m_2015") as shapefile:
    MaskGeom = [feature["geometry"] for feature in shapefile] # Identify the bounding areas of the mask.
# Mask_out = './LatestYearBuffer.tif'

In [6]:
# for filename in os.listdir('NTL/DMSP'):
#     newname = filename.replace('_Masked', '')
#     src =f"{'NTL/DMSP'}/{filename}"  # foldername/filename, if .py file is outside folder
#     dst =f"{'NTL/DMSP'}/{newname}"

#     os.rename(src, dst)

In [23]:
AnnualizedSourceFiles = []
AnnualizedSourceFiles = AnnualizedSourceFiles + [i for i in os.listdir('NTL/SourceFiles/') if i.endswith('tif')]
    
AnnualizedSourceFiles

['F12_1999_avg.tif',
 'F12_1999_cfcvg.tif',
 'F12_1999_ormavg.tif',
 'F14_1999_avg.tif',
 'F14_1999_cfcvg.tif',
 'F14_1999_ormavg.tif',
 'F14_2000_avg.tif',
 'F14_2000_cfcvg.tif',
 'F14_2000_ormavg.tif',
 'F14_2001_avg.tif',
 'F14_2001_cfcvg.tif',
 'F14_2001_ormavg.tif',
 'F14_2002_avg.tif',
 'F14_2002_cfcvg.tif',
 'F14_2002_ormavg.tif',
 'F14_2003_avg.tif',
 'F14_2003_cfcvg.tif',
 'F14_2003_ormavg.tif',
 'F15_2000_avg.tif',
 'F15_2000_cfcvg.tif',
 'F15_2000_ormavg.tif',
 'F15_2001_avg.tif',
 'F15_2001_cfcvg.tif',
 'F15_2001_ormavg.tif',
 'F15_2002_avg.tif',
 'F15_2002_cfcvg.tif',
 'F15_2002_ormavg.tif',
 'F15_2003_avg.tif',
 'F15_2003_cfcvg.tif',
 'F15_2003_ormavg.tif',
 'F15_2004_avg.tif',
 'F15_2004_cfcvg.tif',
 'F15_2004_ormavg.tif',
 'F15_2005_avg.tif',
 'F15_2005_cfcvg.tif',
 'F15_2005_ormavg.tif',
 'F15_2006_avg.tif',
 'F15_2006_cfcvg.tif',
 'F15_2006_ormavg.tif',
 'F15_2007_avg.tif',
 'F15_2007_cfcvg.tif',
 'F15_2007_ormavg.tif',
 'F16_2004_avg.tif',
 'F16_2004_cfcvg.tif',
 'F1

In [24]:
# This codeblock changes each annual population raster's projection (gdal.Warp()), 
# then masks it to within a specified distance of the settlements (rasterio.mask.mask()).

for YearFile in AnnualizedSourceFiles:
    
    InputRasterPath = os.path.join(ProjectFolder, "NTL/SourceFiles", YearFile)
    Sensor = YearFile[:3] # Remove characters in or after position 3 (4th character).
    Year = YearFile[4:8] # Remove characters before position 3, and those in or after position 7.
    # Confusingly, left of : is inclusive and right of : is exclusive. "Slice returns a "substring" text[i:j] from your original String where "i" are the initial index (inclusive) and "j" are the end index (exclusive)."
    
    if YearFile.endswith('_avg.tif') == True:
        IndicType = '_avg'
    elif YearFile.endswith('ormavg.tif') == True:
        IndicType = '_ormavg'
    elif YearFile.endswith('mskavg.tif') == True:
        IndicType = '_mskavg'
    else:
        IndicType = '_cfcvg'
        
    TempOutputName = 'Temp_' + Sensor + '_' + Year + IndicType + ".tif"
    TempOutputPath = os.path.join(ProjectFolder, "NTL", TempOutputName)
    FinalOutputPath = os.path.join(ProjectFolder, "NTL", ''.join(['Msk_', Sensor, '_', Year, IndicType, '.tif']))
    
    if exists(FinalOutputPath): # If we already did both the warp and the mask, the file will be here and we can skip it.
        pass
    else:
        # Reproject to same CRS as settlements.
        InputRasterObject = gdal.Open(InputRasterPath)
        
        if osr.SpatialReference(wkt=InputRasterObject.GetProjection()).GetAttrValue('projcs') != 'Africa_Albers_Equal_Area_Conic':
            Warp = gdal.Warp(TempOutputPath, # Where to store the warped raster
                         InputRasterObject, # Which raster to warp
                         format='GTiff', 
                         options=ProjCRS) # Reproject to Africa Albers Equal Area Conic
            print('Finished gdal.Warp() for %s. %s \n' % (YearFile, time.ctime()))

            Warp = None # Close the files
        else:
            pass
        
        InputRasterObject = None


        # Reclassify as nodata if outside settlement buffer zones.
        if exists(TempOutputPath):
            NewInputPath = TempOutputPath # If we warped the data, then use that file for next step.
        else:
            NewInputPath = InputRasterPath # Otherwise, we must have skipped the warp and can use the source file.
            
        with rasterio.open(NewInputPath) as InputRasterObject:
            MaskedOutputRaster, OutTransform = rasterio.mask.mask(
                InputRasterObject, MaskGeom, crop=True) # Anything outside the mask is reclassed to the raster's NoData value.
            OutMetaData = InputRasterObject.meta.copy()
        print('Finished rasterio.mask.mask() for %s. %s \n' % (YearFile, time.ctime()))
            
        OutMetaData.update({"driver": "GTiff",
                         "height": MaskedOutputRaster.shape[1],
                         "width": MaskedOutputRaster.shape[2],
                         "transform": OutTransform})

        with rasterio.open(FinalOutputPath, "w", **OutMetaData) as dest:
            dest.write(MaskedOutputRaster)
        print('Written to file. %s \n' % time.ctime())
        
    InputRasterObject = None
    
    if exists(TempOutputPath):
        try:  # Finally, remove the intermediate file from disk
            os.remove(TempOutputPath)
        except OSError:
            pass
        print('Removed intermediate file. %s \n' % time.ctime())
    else:
        pass


print('\n \n Finished all years in list. %s' % time.ctime())


 
 Finished all years in list. Fri Jan  6 12:39:54 2023


In [4]:
print(os.listdir('NTL/'))

['Msk_F12_1999_avg.csv', 'Msk_F12_1999_avg.tif', 'Msk_F12_1999_cfcvg.csv', 'Msk_F12_1999_cfcvg.tif', 'Msk_F12_1999_ormavg.csv', 'Msk_F12_1999_ormavg.tif', 'Msk_F14_1999_avg.csv', 'Msk_F14_1999_avg.tif', 'Msk_F14_1999_cfcvg.csv', 'Msk_F14_1999_cfcvg.tif', 'Msk_F14_1999_ormavg.csv', 'Msk_F14_1999_ormavg.tif', 'Msk_F14_2000_avg.csv', 'Msk_F14_2000_avg.tif', 'Msk_F14_2000_cfcvg.csv', 'Msk_F14_2000_cfcvg.tif', 'Msk_F14_2000_ormavg.csv', 'Msk_F14_2000_ormavg.tif', 'Msk_F14_2001_avg.csv', 'Msk_F14_2001_avg.tif', 'Msk_F14_2001_cfcvg.csv', 'Msk_F14_2001_cfcvg.tif', 'Msk_F14_2001_ormavg.csv', 'Msk_F14_2001_ormavg.tif', 'Msk_F14_2002_avg.csv', 'Msk_F14_2002_avg.tif', 'Msk_F14_2002_cfcvg.csv', 'Msk_F14_2002_cfcvg.tif', 'Msk_F14_2002_ormavg.csv', 'Msk_F14_2002_ormavg.tif', 'Msk_F14_2003_avg.csv', 'Msk_F14_2003_avg.tif', 'Msk_F14_2003_cfcvg.csv', 'Msk_F14_2003_cfcvg.tif', 'Msk_F14_2003_ormavg.csv', 'Msk_F14_2003_ormavg.tif', 'Msk_F15_2000_avg.csv', 'Msk_F15_2000_avg.tif', 'Msk_F15_2000_cfcvg.csv', '

In [20]:
AnnualizedSourceFiles = None

### 9.2 Raster values summarized by settlement.
1. Convert each annualized raster to .xyz, 
2. then bring them to vector space and assign their Sett_ID,
3. and finally, aggregate the value as appropriate to the settlement level and save table to file.

XYZ is similar to .csv. Raster cell centers are stored as x and y, and their value is stored as z.

In [8]:
NoDataVal = 0
Settlements = gpd.read_file(r'Results/SETTLEMENTS.gpkg', layer='SETTLEMENTS')[['Sett_ID', 'geometry']]
AllSummaries = pd.DataFrame(Settlements).drop(columns='geometry')

AnnualizedMaskedFiles = [i for i in os.listdir('NTL') if i.endswith('.tif')]
AnnualizedMaskedFiles

['Msk_F12_1999_avg.tif',
 'Msk_F12_1999_cfcvg.tif',
 'Msk_F12_1999_ormavg.tif',
 'Msk_F14_1999_avg.tif',
 'Msk_F14_1999_cfcvg.tif',
 'Msk_F14_1999_ormavg.tif',
 'Msk_F14_2000_avg.tif',
 'Msk_F14_2000_cfcvg.tif',
 'Msk_F14_2000_ormavg.tif',
 'Msk_F14_2001_avg.tif',
 'Msk_F14_2001_cfcvg.tif',
 'Msk_F14_2001_ormavg.tif',
 'Msk_F14_2002_avg.tif',
 'Msk_F14_2002_cfcvg.tif',
 'Msk_F14_2002_ormavg.tif',
 'Msk_F14_2003_avg.tif',
 'Msk_F14_2003_cfcvg.tif',
 'Msk_F14_2003_ormavg.tif',
 'Msk_F15_2000_avg.tif',
 'Msk_F15_2000_cfcvg.tif',
 'Msk_F15_2000_ormavg.tif',
 'Msk_F15_2001_avg.tif',
 'Msk_F15_2001_cfcvg.tif',
 'Msk_F15_2001_ormavg.tif',
 'Msk_F15_2002_avg.tif',
 'Msk_F15_2002_cfcvg.tif',
 'Msk_F15_2002_ormavg.tif',
 'Msk_F15_2003_avg.tif',
 'Msk_F15_2003_cfcvg.tif',
 'Msk_F15_2003_ormavg.tif',
 'Msk_F15_2004_avg.tif',
 'Msk_F15_2004_cfcvg.tif',
 'Msk_F15_2004_ormavg.tif',
 'Msk_F15_2005_avg.tif',
 'Msk_F15_2005_cfcvg.tif',
 'Msk_F15_2005_ormavg.tif',
 'Msk_F15_2006_avg.tif',
 'Msk_F15_2006_

In [10]:
AllSummaries

,Sett_ID
0,1
1,2
2,3
3,4
4,5
...,...
13287,201743
13288,201776
13289,201797
13290,201799


In [11]:
for YearFile in AnnualizedMaskedFiles:
    
### STEP 1: TIF TO XYZ ###
    InputRasterName = os.path.join(ProjectFolder, "NTL", YearFile)
    Sensor = YearFile[4:7]
    Year = YearFile[8:12]
    
    print('Loading data for %s. %s \n' % (YearFile, time.ctime()))
    InputRasterObject = gdal.Open(InputRasterName)
    XYZOutputName = YearFile.replace('.tif', '.xyz') # New file path will be the same as original, but .tif is replaced with .xyz
    XYZOutputPath = r'NTL/{}'.format(XYZOutputName) 
    
    # Create an .xyz version of the .tif
    XYZ = gdal.Translate(XYZOutputPath, # Specify a destination path
                         InputRasterObject, # Input is the masked .tif file
                         format='XYZ', 
                         creationOptions=["ADD_HEADER_LINE=YES"])
    print('Finished gdal.Translate() for %s. %s \n' % (YearFile, time.ctime()))

    # Remove the temporary masked tif file.
#     try:  
#         os.remove(InputRasterName)
#     except OSError:
#         pass
#     print('Removed (or skipped if error) intermediate tif file. %s \n' % time.ctime())
    
    InputRasterObject = None
    XYZ = None # Reload XYZ as a point geodataframe

    
### STEP 2: GENERATE GEODATAFRAME WITH SETT_ID FIELD ###
    InputXYZ = pd.read_table(os.path.join(ProjectFolder, 'NTL', XYZOutputName), delim_whitespace=True)
    InputXYZ = InputXYZ.loc[InputXYZ['Z'] > 0] # Subset to only the cells that contained values of 0 or higher.
    print('Loaded XYZ file as a pandas dataframe, %s. %s \n' % (YearFile, time.ctime()))
    ValObject = gpd.GeoDataFrame(InputXYZ,
                                 geometry = gpd.points_from_xy(InputXYZ['X'], InputXYZ['Y']),
                                 crs = 'ESRI:102022')[['Z', 'geometry']]
    print('Created geodataframe from non-NoData points, %s. %s \n' % (YearFile, time.ctime()))
    del InputXYZ
    
    # Sjoin_nearest: No need to group by ADM this time. 
    ValObject_withID = gpd.sjoin_nearest(ValObject, 
                                    Settlements, 
                                    how='left') # No need for max_distance parameter this time. We've already narrowed down to nearby raster cells.
    
    print('\nJoined settlement ID onto vectorized raster cells for %s. %s \n' % (YearFile, time.ctime()))
    try:
        print(ValObject_withID.sample(10))
    except ValueError:
        pass
    
    del ValObject
    
    # We no longer need the spatial information of the raster values because we have their unique settlement ID.
    ValObject_withID = pd.DataFrame(ValObject_withID).drop(columns='geometry')
    
    ValObject_withID.to_csv(''.join([r'NTL/', YearFile.replace('.tif', '.csv')]))
    print('\nExported as table. %s. %s \n' % (YearFile, time.ctime()))
    
    # Remove the temporary xyz file.
    try:  
        os.remove(os.path.join(ProjectFolder, 'NTL', XYZOutputName))
    except OSError:
        pass
        print('OSError. Skipped.')
    print('Removed (or skipped if error) intermediate xyz file. %s \n' % time.ctime())

    

### STEP 3: AGGREGATE BY SETTLEMENT AND MERGE ONTO SUMMARIES TABLE ###
 
    if YearFile.endswith('_avg.tif') == True:
        IndicType = '_avg'
    elif YearFile.endswith('ormavg.tif') == True:
        IndicType = '_ormavg'
    elif YearFile.endswith('mskavg.tif') == True:
        IndicType = '_mskavg'
    else:
        IndicType = '_cfcvg'

        
    if YearFile.find('cfcvg') == -1:
        # Cell count
        VariableName = ''.join(['NTLct_', Sensor, IndicType, Year])
        ValAggregated = ValObject_withID[
            ValObject_withID['Z'].notna()].groupby(
            'Sett_ID', as_index=False)['Z'].count().rename(columns={"Z": VariableName})
        print('\nCells per settlement counted, year %s. %s \n' % (Year, time.ctime()))
        try:
            print(ValAggregated.sample(10))
        except ValueError:
            pass
        AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
        del VariableName, ValAggregated
    
        # Sum
        VariableName = ''.join(['NTLsum_', Sensor, IndicType, Year])
        ValAggregated = ValObject_withID[
            ValObject_withID['Z'].notna()].groupby(
            'Sett_ID', as_index=False)['Z'].sum().rename(columns={"Z": VariableName})
        print('\nValues summed to settlement level, year %s. %s \n' % (Year, time.ctime()))
        try:
            print(ValAggregated.sample(10))
        except ValueError:
            pass
        AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
        del VariableName, ValAggregated
    
        # Average
        VariableName = ''.join(['NTLavg_', Sensor, IndicType, Year])
        ValAggregated = ValObject_withID[
            ValObject_withID['Z'].notna()].groupby(
            'Sett_ID', as_index=False)['Z'].mean().rename(columns={"Z": VariableName})
        print('\nValues averaged to settlement level, year %s. %s \n' % (Year, time.ctime()))
        try:
            print(ValAggregated.sample(10))
        except ValueError:
            pass
        AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
        print('\nMerged year %s onto latest year settlement feature layer. %s \n' % (Year, time.ctime()))
        del VariableName, ValAggregated
        
        # Maximum
        VariableName = ''.join(['NTLmax_', Sensor, IndicType, Year])
        ValAggregated = ValObject_withID[
            ValObject_withID['Z'].notna()].groupby(
            'Sett_ID', as_index=False)['Z'].max().rename(columns={"Z": VariableName})
        print('\nMax value of available pixels assigned at settlement level, year %s. %s \n' % (Year, time.ctime()))
        try:
            print(ValAggregated.sample(10))
        except ValueError:
            pass
        AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
        print('\nMerged year %s onto latest year settlement feature layer. %s \n' % (Year, time.ctime()))
        del VariableName, ValAggregated
        
        # Minimum
        VariableName = ''.join(['NTLmin_', Sensor, IndicType, Year])
        ValAggregated = ValObject_withID[
            ValObject_withID['Z'].notna()].groupby(
            'Sett_ID', as_index=False)['Z'].min().rename(columns={"Z": VariableName})
        print('\nMin value of available pixels assigned at settlement level, year %s. %s \n' % (Year, time.ctime()))
        try:
            print(ValAggregated.sample(10))
        except ValueError:
            pass
        AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
        print('\nMerged year %s onto latest year settlement feature layer. %s \n' % (Year, time.ctime()))
        del VariableName, ValAggregated

#         # Standard deviation
#         VariableName = ''.join(['NTLsd_', Sensor, IndicType, Year])
#         ValAggregated = ValObject_withID[
#             ValObject_withID['Z'].notna()].groupby(
#             'Sett_ID', as_index=False)['Z'].stdev().rename(columns={"Z": VariableName})
#         print('\nStandard deviation assigned to settlement level, year %s. %s \n' % (Year, time.ctime()))
#         try:
#             print(ValAggregated.sample(10))
#         except ValueError:
#             pass
#         AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
#         print('\nMerged year %s onto latest year settlement feature layer. %s \n' % (Year, time.ctime()))
#         del VariableName, ValAggregated


    else:
        # Average of cloud-free observations
        VariableName = ''.join(['NTLcf_', Sensor, Year])
        ValAggregated = ValObject_withID[
            ValObject_withID['Z'].notna()].groupby(
            'Sett_ID', as_index=False)['Z'].mean().rename(columns={"Z": VariableName})
        print('\nCount of cloud-free observations averaged to settlement level, year %s. %s \n' % (Year, time.ctime()))
        try:
            print(ValAggregated.sample(10))
        except ValueError:
            pass
        AllSummaries = AllSummaries.merge(ValAggregated, how='left', on='Sett_ID')
        print('\nMerged year %s onto latest year settlement feature layer. %s \n' % (Year, time.ctime()))
        del VariableName, ValAggregated
    
    print(AllSummaries.sample(10))

    

print('\n\nFinished. All years masked and assigned their nearest settlement. %s' % time.ctime())

AllSummaries.to_csv(os.path.join(ResultsFolder, 'NTL%sto%s.csv' % (1999, 2015)))
print('Saved to file. %s \n' % time.ctime())

Loading data for Msk_F12_1999_avg.tif. Sun Jan  8 20:29:16 2023 

Finished gdal.Translate() for Msk_F12_1999_avg.tif. Sun Jan  8 20:29:18 2023 

Loaded XYZ file as a pandas dataframe, Msk_F12_1999_avg.tif. Sun Jan  8 20:29:20 2023 

Created geodataframe from non-NoData points, Msk_F12_1999_avg.tif. Sun Jan  8 20:29:20 2023 


Joined settlement ID onto vectorized raster cells for Msk_F12_1999_avg.tif. Sun Jan  8 20:29:22 2023 

         Z                          geometry  index_right  Sett_ID
1078850  5   POINT (-1405558.896 470054.772)         4503    21148
855255   4   POINT (-1534256.429 687177.140)         8074    59171
1057199  4   POINT (-1429197.219 491066.614)         4391    20372
1052469  5   POINT (-1626183.238 495444.081)         2421     7751
992328   3   POINT (-1428321.725 554102.140)         8678    65764
1230213  4   POINT (-1409936.364 322971.878)         1273     3688
893995   3   POINT (-1536882.909 649530.923)         7785    56244
897646   5   POINT (-1495734.719 

Finished gdal.Translate() for Msk_F12_1999_ormavg.tif. Sun Jan  8 20:29:30 2023 

Loaded XYZ file as a pandas dataframe, Msk_F12_1999_ormavg.tif. Sun Jan  8 20:29:32 2023 

Created geodataframe from non-NoData points, Msk_F12_1999_ormavg.tif. Sun Jan  8 20:29:32 2023 


Joined settlement ID onto vectorized raster cells for Msk_F12_1999_ormavg.tif. Sun Jan  8 20:29:35 2023 

                 Z                          geometry  index_right  Sett_ID
1204073   3.111111   POINT (-1419566.791 348361.187)          928     2669
1191350   3.826087   POINT (-1514995.574 360618.095)          254      788
938222    3.100000   POINT (-1468594.423 606631.745)         8938    68652
966816    3.681818   POINT (-1676961.856 578615.956)        12474   196242
1111292   5.130435   POINT (-1400305.936 438537.009)        12693   197698
1011439   4.031250   POINT (-1261977.976 535716.778)         5823    33226
1068737  27.500000   POINT (-1582408.567 479685.200)         3026    12521
1058812   6.303030   PO

Finished gdal.Translate() for Msk_F14_1999_cfcvg.tif. Sun Jan  8 20:29:42 2023 

Loaded XYZ file as a pandas dataframe, Msk_F14_1999_cfcvg.tif. Sun Jan  8 20:29:44 2023 

Created geodataframe from non-NoData points, Msk_F14_1999_cfcvg.tif. Sun Jan  8 20:29:44 2023 


Joined settlement ID onto vectorized raster cells for Msk_F14_1999_cfcvg.tif. Sun Jan  8 20:29:46 2023 

          Z                          geometry  index_right  Sett_ID
1219234  50   POINT (-1556143.764 333477.799)          151      404
1206853  33   POINT (-1352153.798 345734.707)         1424     4086
373642   51  POINT (-1164798.206 1155566.119)        11417   155700
875144   42   POINT (-1475598.370 667916.285)         9525    71570
34075    65  POINT (-1068493.930 1485627.138)        12232   191812
1025711  47   POINT (-1388049.028 521708.884)         5623    28280
898504   39   POINT (-1533380.936 645153.456)         7785    56244
464625   49  POINT (-1180557.088 1067141.284)        11156   145488
1223885  47   P

Finished gdal.Translate() for Msk_F14_2000_avg.tif. Sun Jan  8 20:29:54 2023 

Loaded XYZ file as a pandas dataframe, Msk_F14_2000_avg.tif. Sun Jan  8 20:29:56 2023 

Created geodataframe from non-NoData points, Msk_F14_2000_avg.tif. Sun Jan  8 20:29:56 2023 


Joined settlement ID onto vectorized raster cells for Msk_F14_2000_avg.tif. Sun Jan  8 20:29:58 2023 

         Z                          geometry  index_right  Sett_ID
100764   2  POINT (-1055361.529 1420840.625)        12051   191315
789461   5   POINT (-1552641.791 751088.159)        10152    85422
928262   3   POINT (-1511493.600 616262.173)         7015    49133
1077251  3   POINT (-1227833.732 471805.759)         6107    36358
344749   3  POINT (-1218203.305 1183581.908)        11398   155473
920152   3   POINT (-1512369.094 624141.614)         7015    49133
921046   3   POINT (-1518497.547 623266.120)         7015    49133
1041002  2   POINT (-1410811.857 506825.496)         4602    22135
1025131  4   POINT (-1107015.641

Finished gdal.Translate() for Msk_F14_2000_cfcvg.tif. Sun Jan  8 20:29:59 2023 

Loaded XYZ file as a pandas dataframe, Msk_F14_2000_cfcvg.tif. Sun Jan  8 20:30:01 2023 

Created geodataframe from non-NoData points, Msk_F14_2000_cfcvg.tif. Sun Jan  8 20:30:01 2023 


Joined settlement ID onto vectorized raster cells for Msk_F14_2000_cfcvg.tif. Sun Jan  8 20:30:03 2023 

          Z                          geometry  index_right  Sett_ID
998642   55   POINT (-1422193.271 547973.686)         8467    63148
874196   56   POINT (-1516746.561 668791.778)         7866    56332
1202115  57   POINT (-1556143.764 350112.174)          175      506
890397   47   POINT (-1531629.949 653032.897)         7785    56244
1079766  49   POINT (-1392426.495 469179.279)         3884    19120
1109479  53   POINT (-1409936.364 440287.996)        13114   199357
1053732  59   POINT (-1309254.620 494568.588)         6048    36094
1032387  58   POINT (-1064991.957 515580.430)         6381    38325
465528   70  PO

...

Finished gdal.Translate() for Msk_VII_2014_avg.tif. Sun Jan  8 20:37:56 2023 

Loaded XYZ file as a pandas dataframe, Msk_VII_2014_avg.tif. Sun Jan  8 20:38:04 2023 

Created geodataframe from non-NoData points, Msk_VII_2014_avg.tif. Sun Jan  8 20:38:04 2023 


Joined settlement ID onto vectorized raster cells for Msk_VII_2014_avg.tif. Sun Jan  8 20:38:09 2023 

                Z                         geometry  index_right  Sett_ID
4202943  0.060116  POINT (-1417375.416 497276.596)         3997    19258
3954237  1.483347  POINT (-1430507.979 557686.387)         8372    63000
3233309  0.013697  POINT (-1486540.250 732787.232)        10326    91404
4247575  0.918301  POINT (-1600355.798 486332.793)         3088    12884
4417413  5.508962  POINT (-1403367.349 445184.095)         3884    19120
4002908  0.022289  POINT (-1423066.194 545867.080)         8457    63134
4202490  0.043315  POINT (-1615677.122 497276.596)         3098    12894
3427985  0.149568  POINT (-1460275.123 685510.004) 

Finished gdal.Translate() for Msk_VII_2014_cfcvg.tif. Sun Jan  8 20:38:14 2023 

Loaded XYZ file as a pandas dataframe, Msk_VII_2014_cfcvg.tif. Sun Jan  8 20:38:21 2023 

Created geodataframe from non-NoData points, Msk_VII_2014_cfcvg.tif. Sun Jan  8 20:38:21 2023 


Joined settlement ID onto vectorized raster cells for Msk_VII_2014_cfcvg.tif. Sun Jan  8 20:38:29 2023 

          Z                         geometry  index_right  Sett_ID
4469646  33  POINT (-1414311.151 432489.284)        12680   197684
5156199  43  POINT (-1418250.920 265705.730)          607     1738
3113183  59  POINT (-1220386.966 762116.623)        10566   111590
3844083  42  POINT (-1532066.469 584389.266)        12579   196757
4112333  42  POINT (-1640628.992 519164.202)         6487    45204
3608013  51  POINT (-1535568.486 641734.793)         7367    51710
3647666  48  POINT (-1531628.717 632104.246)        12389   196006
3752233  54  POINT (-1509303.359 606714.624)         8738    66478
3689154  53  POINT (-151

Finished gdal.Translate() for Msk_VII_2015_avg.tif. Sun Jan  8 20:38:50 2023 

Loaded XYZ file as a pandas dataframe, Msk_VII_2015_avg.tif. Sun Jan  8 20:38:57 2023 

Created geodataframe from non-NoData points, Msk_VII_2015_avg.tif. Sun Jan  8 20:38:57 2023 


Joined settlement ID onto vectorized raster cells for Msk_VII_2015_avg.tif. Sun Jan  8 20:39:05 2023 

                Z                         geometry  index_right  Sett_ID
3480051  0.035352  POINT (-1544323.528 672815.192)         7787    56246
3505353  0.086137  POINT (-1511929.872 666686.663)         7856    56321
3840401  1.470202  POINT (-1566211.134 585264.770)         3097    12893
3443943  0.066640  POINT (-1574090.672 681570.235)         9936    79592
3842172  0.082916  POINT (-1579781.449 584827.018)         3201    13019
4292668  7.393425  POINT (-1581532.457 475388.991)         3026    12521
4942740  0.286207   POINT (-989691.604 317798.231)         2051     6337
3705341  0.094406  POINT (-1526813.444 618096.179) 

Finished gdal.Translate() for Msk_VII_2015_cfcvg.tif. Sun Jan  8 20:39:11 2023 

Loaded XYZ file as a pandas dataframe, Msk_VII_2015_cfcvg.tif. Sun Jan  8 20:39:18 2023 

Created geodataframe from non-NoData points, Msk_VII_2015_cfcvg.tif. Sun Jan  8 20:39:18 2023 


Joined settlement ID onto vectorized raster cells for Msk_VII_2015_cfcvg.tif. Sun Jan  8 20:39:26 2023 

          Z                          geometry  index_right  Sett_ID
3750384  52   POINT (-1529877.708 607152.376)         7052    49174
1309435  89  POINT (-1196748.353 1200306.485)        11713   174929
3898058  51   POINT (-1569275.398 571256.703)         3178    12990
3536115  69   POINT (-1455897.602 659244.877)         9199    68949
4844406  44   POINT (-1438825.270 341436.845)          846     2572
3608053  61   POINT (-1518058.401 641734.793)         7015    49133
3768461  71   POINT (-1504925.838 602774.855)         8784    66533
3698154  61   POINT (-1517620.649 619847.187)         7015    49133
3328683  61   P

Saved to file. Sun Jan  8 20:39:45 2023 



In [12]:
# Check contents
AllSummaries.sort_values('NTLsum_F18_avg2012', ascending=False).head(20)

,Sett_ID,NTLct_F12_avg1999,NTLsum_F12_avg1999,NTLavg_F12_avg1999,NTLmax_F12_avg1999,NTLmin_F12_avg1999,NTLcf_F121999,NTLct_F12_ormavg1999,NTLsum_F12_ormavg1999,NTLavg_F12_ormavg1999,...,NTLsum_VII_avg2015,NTLavg_VII_avg2015,NTLmax_VII_avg2015,NTLmin_VII_avg2015,NTLcf_VII2015,NTLct_VII_mskavg2015,NTLsum_VII_mskavg2015,NTLavg_VII_mskavg2015,NTLmax_VII_mskavg2015,NTLmin_VII_mskavg2015
3884,19120,490.0,12541.0,25.593878,63.0,4.0,30.651020,490.0,12061.472411,24.615250,...,16330.026669,8.361509,63.478222,0.295016,62.286738,1922.0,16296.408106,8.478880,63.478222,0.295016
3026,12521,238.0,8572.0,36.016807,63.0,5.0,31.987395,238.0,8318.651921,34.952319,...,7739.773526,8.172939,33.587158,0.947431,54.484688,947.0,7739.773526,8.172939,33.587158,0.947431
7015,49133,881.0,4341.0,4.927355,28.0,3.0,26.981839,881.0,4029.811773,4.574134,...,918.885396,0.258259,7.096574,0.013175,60.500281,421.0,492.525157,1.169893,7.096574,0.086989
7785,56244,667.0,2584.0,3.874063,10.0,3.0,23.746627,667.0,2481.266894,3.720040,...,492.842499,0.184862,3.687119,0.004718,59.092273,203.0,182.149804,0.897290,3.687119,0.101485
3087,12883,65.0,2125.0,32.692308,63.0,10.0,33.276923,65.0,2029.037156,31.215956,...,1221.153314,4.733152,16.127314,0.982590,56.445736,258.0,1221.153314,4.733152,16.127314,0.982590
11165,145497,78.0,1841.0,23.602564,48.0,5.0,46.576923,78.0,1663.207276,21.323170,...,1291.813619,4.320447,25.354900,0.188907,81.063545,297.0,1291.348679,4.347975,25.354900,0.188907
8080,59177,151.0,1180.0,7.814570,22.0,3.0,21.516556,151.0,1039.603595,6.884792,...,622.984079,1.028027,8.337029,0.063526,64.316832,379.0,543.529365,1.434114,8.337029,0.282365
10792,126049,63.0,1102.0,17.492063,32.0,7.0,31.174603,63.0,836.977259,13.285353,...,679.171837,2.553278,19.830303,0.186744,65.812030,253.0,664.300636,2.625694,19.830303,0.212812
9780,74882,51.0,499.0,9.784314,20.0,3.0,32.666667,51.0,446.491629,8.754738,...,373.155921,1.802686,8.955063,0.297555,61.946860,206.0,372.749038,1.809461,8.955063,0.297555
2417,7747,50.0,638.0,12.760000,34.0,6.0,38.460000,50.0,582.733003,11.654660,...,357.070821,1.794326,9.020123,0.282784,54.261307,184.0,351.144288,1.908393,9.020123,0.401295
